In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from jumpadjust import JumpAdjust
from noiseadjust import NoiseAdjust
from volatility import VarianceEstimation
from evaluation import Evaluation
from glob import glob 
dfs = []
for f in glob("./mid_1s/??-??_1s.csv.zst"):
    df = pd.read_csv(f, parse_dates=True, index_col=0)
    dfs.append(df)
df = pd.concat(dfs).sort_index()
df = df.loc["2020":"2020-09-02"]

x = df["BTC-USDT_bitfinex"].ffill().bfill().values


In [2]:
raw_price = df["BTC-USDT_bitfinex"].ffill().bfill().values
results = dict()
for ja in ["truncate_MLE", "truncate", "benchmark"]:
    logprc_ja, _ = JumpAdjust(raw_price).__getattribute__(ja)()
    for na in ["preaveraging", "fracdiff", "benchmark"]:
        logprc_na = NoiseAdjust(logprc_ja).__getattribute__(na)()
        for ev in ["filtration_information", "fractal_dimension"]:
            evaluation = Evaluation(logprc_na).__getattribute__(ev)()
            results[(ja, na, ev)] = evaluation
            print(ja, na, ev, evaluation)

truncate_MLE preaveraging filtration_information 2.5622077269919163e-08
truncate_MLE preaveraging fractal_dimension 1.3546239824041677
truncate_MLE fracdiff filtration_information 2.2961152401338102e-08
truncate_MLE fracdiff fractal_dimension 1.460444099450262
truncate_MLE benchmark filtration_information 1.9758773132318156e-08
truncate_MLE benchmark fractal_dimension 1.4131769914264254
truncate preaveraging filtration_information 3.62516285361102e-08
truncate preaveraging fractal_dimension 1.2093275919251656
truncate fracdiff filtration_information 2.380569650107768e-08
truncate fracdiff fractal_dimension 1.1963514922080505
truncate benchmark filtration_information 2.5586452880486057e-08
truncate benchmark fractal_dimension 1.250992443407212
benchmark preaveraging filtration_information 1.8905088978894405e-08
benchmark preaveraging fractal_dimension 1.3815387082915545
benchmark fracdiff filtration_information 1.3446292122389864e-08
benchmark fracdiff fractal_dimension 1.43524435143852

In [ ]:
# filtration_information : absolute amount of information
# fractal_dimension : 
#  <1.5 for smoother brownian. 
#  =1.5 for pure brownian. 
#  >1.5 for rough process.

pd.Series(results).unstack().sort_values(by="filtration_information")

filtration_information  fractal_dimension
benchmark    benchmark               1.290691e-08           1.399869
             fracdiff                1.344629e-08           1.435244
             preaveraging            1.890509e-08           1.381539
truncate_MLE benchmark               1.975877e-08           1.413177
             fracdiff                2.296115e-08           1.460444
truncate     fracdiff                2.380570e-08           1.196351
             benchmark               2.558645e-08           1.250992
truncate_MLE preaveraging            2.562208e-08           1.354624
truncate     preaveraging            3.625163e-08           1.209328